<a href="https://colab.research.google.com/github/zakarka2006/famcs2024/blob/main/famcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установите библиотеки и скачайте картинки, затем по очереди запускайте блоки с кодом.

In [ ]:
!npm install -g degit
!pip install opencv-python
!pip install easyocr

In [ ]:
!npx degit zakarka2006/famcs2024/images -f ./images

# Letters 1
Начнем с букв: почему-то я сразу начал думать в сторону opencv, как минимум для решения пунктов 2-3, поэтому решение получилось немного "костыльным". Я вырезал отдельные картинки букв размером 60х60 пикселей, но перед этим выкрутил контраст на максимум, чтобы не терять деталей. (Все картинки в папке `./images`)

Как я считаю буквы:
1. Преобразование картинки: функция переводит картинку в ч/б формат и увеличивает контрастность
2. Предобработка изображения: функция загружает изображение и преобразует его в черно-белое. Затем применяется бинаризация с инверсией, чтобы буквы стали белыми на черном фоне.
3. Поиск контуров: на этом этапе я нахожу контуры на бинарном изображении. Найденные контура - это области, содержащие буквы.
4. Подготовка шаблонов для букв: функция создает шаблоны для каждой буквы. Изображения букв предварительно обрабатываются и извлекаются их контуры. Шаблоны сохраняются в словаре для последующего использования.
5. Сопоставление контуров: каждый найденный контур на картинке сопоставляется с шаблоном буквы, в итоге для каждого контура выбирается более подходящий шаблон буквы

Конечно, я пробовал прикрутить ocr для этой задачи, но не все буквы определялись правильно со 100% вероятностью. Поэтому отошел от этой идеи. Работу этого кода проверял вручную на небольших картинках по 8х8 букв, ошибок не нашел, так что должно работать :D.

In [ ]:
import cv2
import numpy as np
from collections import Counter
from PIL import Image, ImageEnhance

Image.MAX_IMAGE_PIXELS = 100000000
def maximize_contrast(image_path):
    image = Image.open(image_path)
    image = image.convert("L")
    enhancer = ImageEnhance.Contrast(image)
    max_contrast_image = enhancer.enhance(10)
    max_contrast_image.save(f"{image_path[:-4]}_contrast.png")

def preprocess_image(image_path):
    # Процесс бинаризации изображения
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image

def find_contours(binary_image):
    # Поиск контуров букв
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def create_letter_templates():
    # Создание шаблонов контуров букв для сравнения
    templates = {}
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        template_path = f"./images/letters/{letter}.png"
        template_image = preprocess_image(template_path)
        contours = find_contours(template_image)
        if contours:
            templates[letter] = contours
    return templates

def match_letters(image, templates):
    # Перебор найденных контуров
    detected_letters = []
    contours = find_contours(image)
    for contour in contours:
        # Выбор наиболее подходящего шаблона буквы
        x, y, w, h = cv2.boundingRect(contour)
        letter_image = image[y:y+h, x:x+w]
        max_match = float("inf")
        matched_letter = ""
        for letter, template_contours in templates.items():
            for template_contour in template_contours:
                match = cv2.matchShapes(template_contour, contour, cv2.CONTOURS_MATCH_I3, 0.0)
                if match < max_match:
                    max_match = match
                    matched_letter = letter
        detected_letters.append(matched_letter)
    return detected_letters

def count_letters():
    # Подготовка картинки
    maximize_contrast("./images/letters.png")
    binary_image = preprocess_image("./images/letters_contrast.png")
    templates = create_letter_templates()
    detected_letters = match_letters(binary_image, templates)

    letter_count = Counter(detected_letters)
    sorted_letters = sorted(letter_count.items())
    total_count = 0

    for letter, count in sorted_letters:
        if not letter.isalpha():
            continue
        total_count += count
        print(f"{letter}: {count}")

    print(f"Total: {total_count}")

count_letters()

# Letters 2
Самый простой пункт, по моему мнению, т.к. все просто сводится к нахождению контуров букв и отрисовки рамок вокруг них.

Итоговая картинка лежит в `./images/letters_rectangles.png`



In [ ]:
import cv2
import numpy as np
import time

def add_bounding_boxes_letters():
    padding = 2
    thickness = 3
    # Подготовка картинки
    maximize_contrast("./images/letters.png")
    img = cv2.imread("./images/letters_contrast.png")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Применяем бинарное пороговое преобразование, чтобы создать бинарное изображение (символы белые, фон черный)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Определяем ядро для эрозии
    kernel = np.ones((3, 3), np.uint8)

    # Применяем эрозию к изображению, чтобы уменьшить шум и сделать символы более отчетливыми
    img_erode = cv2.erode(thresh, kernel, iterations=1)

    # Находим контуры на эрозированном изображении (каждый контур соответствует символу)
    contours, _ = cv2.findContours(img_erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    output = cv2.imread("./images/letters.png")
    padding += thickness
    for contour in contours:
        # Расчет размеров рамки и ее отрисовка
        x, y, w, h = cv2.boundingRect(contour)
        x_pad = max(x - padding - 1, 0)
        y_pad = max(y - padding - 1, 0)
        dx = (0 - (x - padding - 1)) if x_pad == 0 else 0 # для букв возле левой границы
        w_pad = w + 2*padding + 1 - dx
        h_pad = h + 2*padding + 1

        border_color = (0, 0, 0)
        cv2.line(output, (x_pad, y_pad), (x_pad + w_pad, y_pad), border_color, thickness)
        cv2.line(output, (x_pad, y_pad + h_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        cv2.line(output, (x_pad + w_pad, y_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        if dx == 0:
            cv2.line(output, (x_pad, y_pad), (x_pad, y_pad + h_pad), border_color, thickness)

    cv2.imwrite("./images/letters_rectangles.png", output)
    print("Done")

add_bounding_boxes_letters()

# Letters 3
Можно немного изменить второй код и вместо рамок рисовать фон под буквами
Как работает:
1. Поиск контуров
2. Выбор самого встречающегося цвета в пределах контура
3. Наложение прямоугольной рамки
4. Наложение буквы на итоговую картинку с помощью маски

Итоговая картинка лежит в `./images/letters_backgrounds.png`


In [ ]:
import cv2
import numpy as np
import time

def add_background_letters():
    maximize_contrast("./images/letters.png")
    img = cv2.imread("./images/letters_contrast.png")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    margin = 3

    imgc = cv2.imread("./images/letters.png")
    output = imgc.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        letter_color = np.mean(output[y:y+h, x:x+w], axis=(0, 1))

        background_color = [255 - int(c) for c in letter_color]

        cv2.rectangle(output, (x-margin, y-margin), (x+w+margin, y+h+margin), background_color, -1)

        letter_region = imgc[y:y+h, x:x+w]
        mask = binary[y:y+h, x:x+w]
        mask_inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(output[y:y+h, x:x+w], output[y:y+h, x:x+w], mask=mask_inv)
        fg = cv2.bitwise_and(letter_region, letter_region, mask=mask)
        output[y:y+h, x:x+w] = cv2.add(bg, fg)

    cv2.imwrite("./images/letters_backgrounds.png", output)
    print("Done")

add_background_letters()

# Words 5, 6
Так как мое "чудесное" решение зависит от нахождения контуров, то пришлось придумывать что-то другое для слов, потому что они расположены хаотично на картинке. Ввиду недостатка времени пришлось костылять: сначала пробовал искать контуры для слов только с помощью OpenCV, но еще во время поиска инстументов для распознавания символов наткнулся на библиотеку `easyocr`, так что первоначальное определение контуров она взяла на себя, а затем я с помощью OpenCV доделывал работу. Идея такая:
1. Получить список контуров с помощью `easyocr`
2. Отсортировать слова на 2 типа: вертикальные и горизонтальные
3. Внутри этих контуров уже искать настоящие контуры слов
4. Финальные манипуляции с картинкой

Разделить и почистить код времени не было, так что тут все 3 пункта в одной куче.

Итоговые картинки: `./images/words_rectangles.png`, `./images/words_backgrounds.png`

P.S.: идеального результата добиться не удалось, опять же, из-за хаотичного расположения слов, но по бОльшая часть слов определяется хорошо(если контур нашелся, то с рамкой и фоном все ок будет)

P.S. 2: а еще есть косяк с разделением слова на 2 части, т.е. есть линия посреди слова :(

P.S. 3: вообще, я пытался прикрутить определение слов(4ый пункт), в функции, где сохраняются координаты слов, нужно разбивать слова на буквы и проделывать те же действия, что и в первом пункте. Но что-то пошло не так, поэтому не стал включать в это решение.

In [ ]:
import easyocr
import cv2
import random
import PIL

reader = easyocr.Reader(["en"])

In [192]:
# Выполняется около 3-х минут
image_path = "./images/words.png"
maximize_contrast(image_path)
image = cv2.imread("./images/words_contrast.png")

# Получение контуров "групп" слов

bounds = reader.detect(image_path, mag_ratio=0.7, add_margin=0) # в параметрах убрал отступы и уменьшил коэффициент увеличения изображения

In [ ]:
def draw_boxes(image, bounds, width=2):
    # Не успел убрать старый функционал, но функция рассортировывает контуры слов на горизонтальные и вертикальные
    horizontal_bounds = []
    vertical_bounds = []
    for bound in bounds[0][0]:
        x_min, x_max, y_min, y_max = bound
        top_left = (x_min, y_min)
        bottom_right = (x_max, y_max)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        if (y_max-y_min > x_max-x_min):
            vertical_bounds.append(bound)
            # vert
            color = (0, 255, 0)
        else:
            horizontal_bounds.append(bound)
            # hor
            color = (0, 0, 255)
        cv2.rectangle(image, top_left, bottom_right, color, width)

    # Сортировка контуров
    horizontal_bounds = sorted(horizontal_bounds, key=lambda x: (x[1], x[0]))
    vertical_bounds = sorted(vertical_bounds, key=lambda x: (x[1], x[0]))
    return image, horizontal_bounds, vertical_bounds

image_with_boxes, horizontal_bounds, vertical_bounds = draw_boxes(image, bounds)
image = cv2.imread(image_path)

def find_words(img, x_min, y_min, hor=True):
    global final
    words = []

    # Преобразование изображения в градации серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Применение размытия для уменьшения шума
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Адаптивное пороговое преобразование для выделения символов на изображении
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)

    # Определение структуры для морфологической операции
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (12, 3))

    # Применение дилатации для расширения белых областей (букв)
    dilation = cv2.dilate(thresh, rect_kernel, iterations=1)

    # Поиск контуров на изображении после дилатации
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                           cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if (hor):
            words.append((x_min + x, y_min + y, w, h))
        else:
             # Если слова вертикальные, корректируем координаты и размеры
            words.append((x_min + y, y_max - x - w, h, w))

    return words

# Списки для хранения координат слов
words_hor = []
words_vert = []
crop = []
for bound in horizontal_bounds:
    x_min, x_max, y_min, y_max = bound
    crop = image[y_min:y_max, x_min:x_max]
    words_hor.extend(find_words(crop, x_min, y_min))

for bound in vertical_bounds:
    x_min, x_max, y_min, y_max = bound
    crop = cv2.rotate(image[y_min:y_max, x_min:x_max], cv2.ROTATE_90_CLOCKWISE)
    words_vert.extend(find_words(crop, x_min, y_min, False))

def draw_rectangles_words():
    thickness = 2
    padding = 4
    img = cv2.imread(image_path)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    margin = 3
    imgc = img.copy()
    background_img = img.copy()
    for word in words_hor:
        x, y, w, h = word
        if (w < 80 or h > 63 or h < 30):
            continue
        x_pad = max(x - padding - 1, 0)
        y_pad = max(y - padding - 1, 0)
        dx = (0 - (x - padding - 1)) if x_pad == 0 else 0
        w_pad = w + 2*padding + 1 - dx
        h_pad = h + 2*padding + 1

        border_color = (0, 0, 0)
        cv2.line(img, (x_pad, y_pad), (x_pad + w_pad, y_pad), border_color, thickness)
        cv2.line(img, (x_pad, y_pad + h_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        cv2.line(img, (x_pad + w_pad, y_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        if dx == 0:
            cv2.line(img, (x_pad, y_pad), (x_pad, y_pad + h_pad), border_color, thickness)

        letter_color = np.mean(background_img[y:y+h, x:x+w], axis=(0, 1))
        background_color = [255 - int(c) for c in letter_color]
        cv2.rectangle(background_img, (x-margin, y-margin), (x+w+margin, y+h+margin), background_color, -1)

        letter_region = imgc[y:y+h, x:x+w]
        mask = binary[y:y+h, x:x+w]
        mask_inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(background_img[y:y+h, x:x+w], background_img[y:y+h, x:x+w], mask=mask_inv)
        fg = cv2.bitwise_and(letter_region, letter_region, mask=mask)
        background_img[y:y+h, x:x+w] = cv2.add(bg, fg)

    for word in words_vert:
        x, y, w, h = word
        if (h < 80 or w > 63 or w < 30):
            continue
        x_pad = max(x - padding - 1, 0)
        y_pad = max(y - padding - 1, 0)
        dx = (0 - (x - padding - 1)) if x_pad == 0 else 0
        w_pad = w + 2*padding + 1 - dx
        h_pad = h + 2*padding + 1

        border_color = (0, 0, 0)
        cv2.line(img, (x_pad, y_pad), (x_pad + w_pad, y_pad), border_color, thickness)
        cv2.line(img, (x_pad, y_pad + h_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        cv2.line(img, (x_pad + w_pad, y_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        if dx == 0:
            cv2.line(img, (x_pad, y_pad), (x_pad, y_pad + h_pad), border_color, thickness)

        letter_color = np.mean(background_img[y:y+h, x:x+w], axis=(0, 1))
        background_color = [255 - int(c) for c in letter_color]
        cv2.rectangle(background_img, (x-margin, y-margin), (x+w+margin, y+h+margin), background_color, -1)

        letter_region = imgc[y:y+h, x:x+w]
        mask = binary[y:y+h, x:x+w]
        mask_inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(background_img[y:y+h, x:x+w], background_img[y:y+h, x:x+w], mask=mask_inv)
        fg = cv2.bitwise_and(letter_region, letter_region, mask=mask)
        background_img[y:y+h, x:x+w] = cv2.add(bg, fg)

    return img, background_img

rect_img, background_img = draw_rectangles_words()
cv2.imwrite("./images/words_rectangles.png", rect_img)
cv2.imwrite("./images/words_backgrounds.png", background_img)

# Итог

С буквами все прошло хорошо, но для более сложных задач (по типу поиска слов, когда они хаотично разбросаны) нужно придумывать более элегантное решение. Ну а развивать это решение для слов не имеет смысла :D.

P.S. Надеюсь, что ход моих мыслей был понятен
